### Short list of objectives

* Import pdf of Proposed Rule XXXXXXX
* Query rule for baseline responses 
* Fine-tune embedding model, recheck responses
* Summarize rule

In [ ]:
# Set up ollama
!apt-get update && apt-get install tmux vim -y
!pip3 install llama-index llama-parse llama_deploy llama-index-llms-huggingface llama-index-embeddings-huggingface llama-index-llms-ollama llama-index-embeddings-ollama llama-index-vector-stores-neo4jvector llama-index-graph-stores-neo4j llama-index-finetuning llama-index-utils-workflow llama-index-readers-file
!pip3 install sentencepiece protobuf evaluate rouge_score absl-py tensorboardX bitsandbytes peft accelerate python-dotenv graspologic fpdf2
!pip3 install flash-attn --no-build-isolation
!curl -fsSL https://ollama.com/install.sh | sh


In [ ]:
# Set up Neo4j
!apt install dialog apt-utils -y 
!wget -O - https://debian.neo4j.com/neotechnology.gpg.key | gpg --dearmor -o /etc/apt/keyrings/neotechnology.gpg
!echo 'deb [signed-by=/etc/apt/keyrings/neotechnology.gpg] https://debian.neo4j.com stable latest' | tee -a /etc/apt/sources.list.d/neo4j.list
!apt list -a neo4j
!add-apt-repository universe -y
!apt install neo4j=1:5.23.0 -y
!echo "neo4j-enterprise neo4j/question select I ACCEPT" | debconf-set-selections
!echo "neo4j-enterprise neo4j/license note" | debconf-set-selections
!apt install openjdk-17-jre -y
!cd /var/lib/neo4j/plugins/ && wget https://github.com/neo4j/apoc/releases/download/5.23.0/apoc-5.23.0-core.jar

In [ ]:
# setting up llama.cpp 22m 32s
!cd /workspace/repos/ && git clone https://github.com/ggerganov/llama.cpp
!cd /workspace/repos/llama.cpp && git pull && make clean && LLAMA_CUDA=0 make
!chmod 755 /workspace/repos/llama.cpp/requirements.txt && pip3 install -r /workspace/repos/llama.cpp/requirements.txt 

In [ ]:
import torch
# from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

save_dir = "/workspace/data/compliance/proposal_embedworking"

# tokenizer_name = "dunzhang/stella_en_1.5B_v5"
tokenizer_name = "Qwen/Qwen2-1.5B"
model_name = "/workspace/data/compliance/rule_proposal_embedding_model_gpt"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             return_dict=True,
                                             torch_dtype=torch.float16,
                                             device_map="auto",
                                             )
print('Saving model')
model.save_pretrained(save_dir)
print('Saving tokenizer')
tokenizer.save_pretrained(save_dir)

In [ ]:
!python3 /workspace/repos/llama.cpp/convert_hf_to_gguf.py /workspace/data/compliance/proposal_embed
!cd /workspace/data/compliance/proposal_embed && echo 'FROM "/workspace/data/compliance/proposal_embed/Proposal_Embed-1.8B-F16.gguf"' >> Modelfile && ollama create proposal_embed -f Modelfile

    #FROM "/workspace/data/compliance/proposal_embed/Proposal_Embed-1.8B-F16.gguf"
# then run in terminal
    # ollama create proposal_embed -f Modelfile

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# TODO: notify if ollama server is running with model loaded
import subprocess, os
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI as LOpenAI
from dotenv import load_dotenv
load_dotenv('/workspace/repos/agentic-ai/.env')

# model_name, ctx_len = "gpt-4o-2024-08-06", 128000
model_name, ctx_len = "llama3.1:8b-instruct-q8_0", 128000
# model_name, ctx_len = "solar-pro:latest", 128000
# model_name, ctx_len = "hermes3:8b", 128000
# model_name, ctx_len = "gemma2:latest", 8192


if "gpt-4o" in model_name:
    openai_key = os.getenv("OPENAI_API_KEY")
    os.environ["OPENAI_API_KEY"] = openai_key
    
    print(f"Using OpenAI {model_name}...")
    llm = LOpenAI(model=model_name, max_tokens=8000)
else:
    try: 
        print("Pulling Ollama model...")
        sub_out = subprocess.run(['ollama', 'pull', model_name], capture_output=True, text=True)
    except Exception as e: 
        print(f"Error pulling model: Is the Ollama server running?\n{e}")
    
    addtion_kwargs = {"max_new_tokens": 8000}
    # system_prompt = "You are an expert at answering questions about rules and regulations regarding Title 17—Commodity and Securities Exchanges: CHAPTER II—SECURITIES AND EXCHANGE COMMISSION. Please provide a summary of the following text, and cite any sections, rules, acts or laws (e.g. § 230.503, § 240.13a-15, Act (15 U.S.C. 781), Investment Company Act of 1940) from context that support the answer. Be detailed in your response."
    llm = Ollama(model=model_name, url="http://127.0.0.1:11434", context_window=ctx_len, model_type="chat", is_function_calling_model=False, 
                 request_timeout=4000.0, **addtion_kwargs) #, system_prompt=system_prompt) additional_kwargs=addtion_kwargs,
    print(llm.metadata)

Settings.llm = llm

## Import data

In [ ]:
# SEC rules and regulations
!cd /workspace/data && curl -X GET "https://www.ecfr.gov/api/versioner/v1/full/2024-07-23/title-17.xml?chapter=II" -H "accept: application/xml" > title-17.xml

In [4]:
import xml.etree.ElementTree as ET
from sec_utils import get_tree_data

# Path to your XML file
xml_file_path = '/workspace/data/title-17.xml'

# Parse the XML file
tree = ET.parse(xml_file_path)

# Get the root element of the XML document
root = tree.getroot()

sec_data = get_tree_data(root)



In [5]:
from sec_utils import get_metadata
from llama_index.core import Document

documents = [Document(text=t, 
                          text_template='{metadata_str}\n\n{content}',
                          metadata=get_metadata(m, t, metadata={"section":None, "description":None, "mentioned_sections":None})) \
                            for m,t in sec_data.items()]

In [ ]:
import os, re
from llama_index.readers.file import PDFReader
# from llamaindex_data_utils import extract_text_from_pdf
from llama_index.core import Document
from utils import fix_hyphenated_words
from dotenv import load_dotenv
load_dotenv('/workspace/repos/agentic-ai/.env')

access_token = os.getenv('HF_TOKEN')
llama_api_key = os.getenv('LLAMA_API_KEY')

parser = PDFReader(return_full_document=True)
documents_proposal = parser.load_data("/workspace/data/compliance/FSI_Rule_proposal.pdf")

# fixes poorly formatted hyphenated words
doc_text = documents_proposal[0].text
documents_proposal[0].text = fix_hyphenated_words(doc_text)

step=10000
start=0
track=[]
for i in range(1, 18):  # Adjust the range as needed
    # Regular expression pattern to match the specific number preceded by \n \n or \n\n and followed by a space
    if i == 1:
        pattern = rf'(\n \n|\n\n| \n)({i})( )'
    else:
        pattern = rf'(\n \n|\n\n)({i})( )'
    text_slice = doc_text[start:start+step]
    # Check if the pattern exists in the doc_text
    if re.search(pattern, text_slice):
        # Replace the matching patterns with an empty string
        matches = list(re.finditer(pattern, text_slice))
        # for mat in range(len(matches)):
        #     if start > matches[mat].span()[0]:
        #         continue
        #     else:
        #         break
        # if matches[mat].span()[0] < start:
        #     continue
        # else:
        print(f"Found {i} {matches[0].span()}")
        track.append(i)
        bingo=matches[0].span()
        text_slice = text_slice[:bingo[0]]+"\n\n"+text_slice[bingo[1]:]
        doc_text = doc_text[:start] + text_slice + doc_text[start+step:]
        start = bingo[1]
    else:
        start += step

# documents_proposal[0].text = doc_text
# pattern = r'(?<=\n)([IVX]+\..*?)(?=\n[IVX]+\.\s|\Z)'
# sections=[]
# sections.append("Proposal Information"+documents_proposal[0].text.split("I. Background A. Sta")[0])
# sections += re.findall(pattern, documents_proposal[0].text, re.DOTALL)
# section4_5split = sections[4].split("V. Paperwork Reduction Act")
# sections[4]=section4_5split[0]
# sections = sections[:5]+["V. Paperwork Reduction Act"+section4_5split[-1]]+sections[5:]
# for shead in [x.split('\n')[0].strip() for x in sections]:
#     print(shead)

# # pdf_urls = ["/workspace/data/compliance/FSI_Factsheet.pdf", "/workspace/data/compliance/FSI_Press_Release.pdf", "/workspace/data/compliance/FSI_Rule_proposal.pdf"]
# # descriptions = ["Factsheet about newly proposed SEC rule.", "Press release regarding newly proposed SEC rule.", "The full text of the newly proposed SEC rule."]
# # documents_proposal = extract_text_from_pdf(pdf_urls, llama_api_key, llamaparse_kwargs={"split_by_page":False}, save_json_path=None)
# documents_proposal = [Document(text=t, 
#                           text_template='{metadata_str}\n\n{content}') for t in sections]

# # add metadata to the documents_proposal
# for i in range(len(documents_proposal)):
#     documents_proposal[i].metadata["section"] = documents_proposal[i].text.split("\n")[0].strip()
    

In [ ]:
print(docs[0].text[19000:23000])

In [11]:
parser = PDFReader(return_full_document=True)
docs = parser.load_data("/workspace/data/compliance/FSI_Rule_proposal.pdf")

In [ ]:
import re
test="""Something here.

5 ksjdkfjlksdjf

4 financing of terrorism, and other illicit financ e activity, and to safeguard the national security of 
the United States .1  The Secretary of the Treasury (“ the Secretary”) delegated the authority to 
implement, administer, and enforce the BSA and its implementing regulations to the Director of 
FinCEN .2 
person’s identity, including name, address, and other identifying information; and (C) consulting lists of known or suspected terrorists or terrorist organizations provided to the financial institution by any government agency to determine whether a person seeking to open an account appears on any such list.”
3 These programs are referred to as Customer Identification Program s 
 
1   See 31 U.S.C. 5311. Certain parts of the Currency and Foreign Transactions Reporting Act, as amended , 
and other statutes relating to the subject matter of that Act, have come to be referred to as the BSA. The 
BSA is codified at 12 U.S.C. 1829b, 12 U.S.C. 1951- 1960, and 31 U.S.C. 310, 5311- 5314, 5316- 5336, 
including notes thereto, with implementing regulations at 31 CFR chapter X.  
2 See Treasury Order 180 –01, paragraph 3(a) (Jan. 14, 2020), available at https://home.treasury.gov/about/
general -information/orders -and-directives/treasury -order -180-01. 
3 31 U.S.C. 5318(l)(2).  
 
5 (“CIP s”) and are long-standing, foundational components of a financial institution’s  anti-money 
laundering program.   
As enacted , section 326 applies to all “financial institutions .”  This term is  defined 
broadly in the BSA to encompass a variety of entities, including commercial banks; agencies, 
and branches of foreign banks in the United States ; thrift  institutions , credit unions, and private 
banker s; trust companies; securities brokers and dealers registered with the Commission ; 
investment companies; futures commission merchants ; insurance companies ; travel agencies ; 
pawnbrokers; dealers in precious metals , stones, and jewels; check -cashers ; certain  casinos; and 
telegraph companies, among others.4  The BSA also grants authority to the Secretary  to define, 
by regulation, additional types of businesses as financial institutions where the Secretary  
determines that such businesses engage in any activity “similar to, related to, or a substitute for” those in which any of the businesses listed in the statutory definition are authorized to engage.

As part of the implementation, administration , and enforcement of the BSA , this authority has 
"""
test=docs[0].text


step=20000
start=0
for i in range(1, 7):  # Adjust the range as needed
    # Regular expression pattern to match the specific number preceded by \n \n or \n\n and followed by a space
    if i == 1:
        pattern = rf'(\n \n|\n\n| \n)({i})( )'
    else:
        pattern = rf'(\n \n|\n\n)({i})( )'
    text_slice = test[start:start+step]
    # Check if the pattern exists in the test
    if re.search(pattern, text_slice):
        print(f"Found {i}")
        # Replace the matching patterns with an empty string
        matches = list(re.finditer(pattern, text_slice))
        # if len(matches) > 0:
        print(matches)
        bingo=matches[0].span()
        text_slice = text_slice[:bingo[0]]+"\n\n"+text_slice[bingo[1]:]
        test = test[:start] + text_slice + test[start+step:]
        start = bingo[1]
    else:
        start += step

# print(test)

In [156]:
pattern = rf'(\n \n|\n\n)({1})( )'
matches = list(re.finditer(pattern, test))

In [ ]:
test[:3000]

### Create Distillation Datasets

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(chunk_size=300, chunk_overlap=50)
nodes = parser.get_nodes_from_documents(documents_proposal, show_progress=True)
len(nodes)

In [ ]:
system_prompt = 'Context information is below.\n\n---------------------\n{context_str}\n---------------------\n\nGiven the context information and no prior knowledge, generate {num_questions_per_chunk} questions based on the below query.\n\nYou are a FINRA certified Compliance Specialist that writes exams for firm compliance professionals. Your task is to write precise questions for an upcoming Compliance Officer certification examination. The questions should be diverse across the context with no multiple choice. Restrict the questions to the context information provided."\n'

In [ ]:
from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset
from llama_index.llms.openai import OpenAI
import random

output_path = "/workspace/data/train_dataset_proposal.json"
if not os.path.exists(output_path):
    rand_index = random.sample(range(len(nodes)), len(nodes))
    train_perc=1.0
    train_size = int(len(rand_index)*train_perc)

    train_dataset = generate_qa_embedding_pairs(
        qa_generate_prompt_tmpl = system_prompt,
        num_questions_per_chunk=5,
        save_every=20,
        output_path=output_path,
        llm=llm, 
        nodes=[nodes[x] for x in rand_index[:train_size]],
        verbose=False
    )
    train_dataset.save_json(output_path)

    # val_dataset = generate_qa_embedding_pairs(
    #     qa_generate_prompt_tmpl = system_prompt,
    #     num_questions_per_chunk=5,
    #     save_every=500,
    #     output_path="/workspace/data/val_dataset.json",
    #     llm=llm, 
    #     nodes=[nodes[x] for x in rand_index[train_size:]],
    #     verbose=False
    # )
    # # assert len(val_dataset.relevant_docs) == len(val_dataset.queries)
    # assert (np.unique(list(val_dataset.relevant_docs.values()), return_counts=True)[1]==5).all()
    # val_dataset.save_json("/workspace/data/val_dataset.json")



### Pretrain Embeddings

In [ ]:
%load_ext autoreload
%autoreload 2

#####################
# in llama_index/embeddings/huggingface/base need to add "show_progress_bar=False" to 204, 213
#####################

import os, json

from llama_index.finetuning import EmbeddingAdapterFinetuneEngine, SentenceTransformersFinetuneEngine
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings


train_path = "/workspace/data/train_dataset_proposal.json"
val_path = None #"/workspace/data/val_dataset.json"
if os.path.exists(train_path):
    train_dataset = EmbeddingQAFinetuneDataset.from_json(train_path)
if val_path is not None and os.path.exists(val_path):
    val_dataset = EmbeddingQAFinetuneDataset.from_json(val_path)
else:
    val_dataset = None

embed_model_name = "dunzhang/stella_en_1.5B_v5" #7b params
print("loading embed model...")

finetune_engine = SentenceTransformersFinetuneEngine(
    train_dataset,
    embed_model_name,
    batch_size=6,
    model_output_path="/workspace/data/rule_proposal_embedding_model",
    val_dataset=val_dataset,
    epochs=4,
    show_progress_bar=True,
    # can optionally pass along any parameters that go into `train_model`
    # optimizer_class=torch.optim.SGD,
    # optimizer_params={"lr": 0.0001, "weight_decay": 0.01}
)


# finetune_engine = EmbeddingAdapterFinetuneEngine(
#     train_dataset,
#     base_embed_model,
#     batch_size=10,
#     model_output_path="/workspace/data/adapter_model",
#     # val_dataset=val_dataset,
#     epochs=4,
#     verbose=False,
#     # can optionally pass along any parameters that go into `train_model`
#     # optimizer_class=torch.optim.SGD,
#     # optimizer_params={"lr": 0.001}
# )

In [ ]:
finetune_engine.finetune(**{"lr": 0.001, "weight_decay": 0.0001})

In [6]:
import gc, torch

# Clear memory
del finetune_engine, train_dataset
gc.collect()
torch.cuda.empty_cache()

## Embedding Model

In [ ]:

import os
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

embed_model_name = "dunzhang/stella_en_1.5B_v5" #7b params
finetuned_embed_model_name = "/workspace/data/compliance/rule_proposal_embedding_model_gpt"
print("loading embed model...")
proposal_embed_model = HuggingFaceEmbedding(model_name=finetuned_embed_model_name)
rules_embed_model = HuggingFaceEmbedding(model_name=embed_model_name)

# Settings.embed_model = embed_model
# Settings.chunk_size = 300
# Settings.chunk_overlap = 50

## GraphRAG

In [6]:
from typing import Literal

entities = Literal[#"PROPOSED_RULE", 
                   "ACTS",
                   "SECTIONS",
                   "RULES"
                   #"AMENDMENTS"
]

relations = Literal[
    "MENTIONS",
    #"CHANGES",
    # "AMENDS", 
    "REFERS_TO"
]

validation_schema = {
    # "Proposed_Rules": ["CHANGES", "AMENDS"],
    "Acts": ["MENTIONS","REFERS_TO"],
    "Sections": ["MENTIONS","REFERS_TO"],
    "Rules": ["MENTIONS","REFERS_TO"],
    # "Amendments": ["REFERS_TO", "AMENDS"],
}


In [ ]:
from rag_utils import set_neo4j_password, add_lines_to_conf
set_neo4j_password('bewaretheneo')
# add_lines_to_conf()

###### START NEO4J SERVER ######


In [ ]:
# GraphRAG Database
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor, SimpleLLMPathExtractor
from rag_utils import create_neo4j_graph_store, create_neo4j_graphrag, neo4j_query, dump_neo4j_database
import random
from dotenv import load_dotenv
load_dotenv('/workspace/repos/agentic-ai/.env')

access_token = os.getenv('HF_TOKEN')
llama_api_key = os.getenv('LLAMA_API_KEY')

graph_idx_persist_dir = "/workspace/data/compliance/graph_idx_testfull"
graph_store_persist_dir= None #"/workspace/data/graph_store"

Settings.chunk_size = 500
Settings.chunk_overlap = 20

# kg_extractor = SchemaLLMPathExtractor(
#     llm=llm,
#     possible_entities=entities,
#     possible_relations=relations,
#     kg_validation_schema=validation_schema,
#     strict=False,  # if false, will allow triples outside of the schema``
#     num_workers=10,
#     max_triplets_per_chunk=10,
# )


# extract_prompt = "You are an expert compliance officer with vast knowledge of SEC Title 17 Chapter II. Your job is to read each section and link mentions of other sections, rules, or acts (e.g. § 230.503, § 240.13a-15, Act (15 U.S.C. 781), Investment Company Act of 1940) mentioned. If there are no mentions of other sections, rules, or acts, return an empty list."
# extract_prompt = "You are an expert compliance officer with vast knowledge of SEC Title 17 Chapter II. Your job is to read each section and link other sections, rules, and acts mentioned. What sections, rules, and acts (e.g. § 230.503, § 240.13a-15, Act (15 U.S.C. 781), Investment Company Act of 1940) are mentioned in the content? If there are no mentions, return an empty list."
llm.is_function_calling_model = False
extract_prompt = "You are an expert compliance officer with vast knowledge of SEC Title 17 Chapter II. Your job is to find semantic, referential, and literal relationships between the sections. If there are no relationships, return an empty list."
kg_extractor = SimpleLLMPathExtractor(
        extract_prompt=extract_prompt,
        llm=llm,
        max_paths_per_chunk=10,
        num_workers=6,
    )

# random.shuffle(documents)

print("Creating graph store...")
graph_store = create_neo4j_graph_store(neo_url="bolt://localhost:7687", 
                                       password=os.getenv("NEO4J_PWD"), 
                                       config={"connection_timeout": 1000, "connection_acquisition_timeout": 1000, "max_connection_pool_size": 1000})

if not os.path.exists(graph_idx_persist_dir):
    print("Deleting all nodes and relationships...")
    neo4j_query(graph_store, query="""MATCH n=() DETACH DELETE n""")

print("Creating graphrag index...")
graph_index = create_neo4j_graphrag(documents, llm, rules_embed_model, kg_extractor, graph_store, graph_idx_persist_dir=graph_idx_persist_dir, graph_store_persist_dir=graph_store_persist_dir, similarity_top_k=3)

dump_neo4j_database('neo4j', '/workspace/data/')

In [ ]:
query_engine = graph_index.as_query_engine()

In [7]:
neo4j_query(graph_store, query="""MATCH n=() DETACH DELETE n""")
graph_store.close()

## RAG

In [ ]:
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="llama2",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

In [ ]:
# Vector Database RAG
from llama_index.core.postprocessor import LLMRerank
from rag_utils import create_llama_vector_index_rag

Settings.chunk_size = 300
Settings.chunk_overlap = 20
vector_index = create_llama_vector_index_rag(llm, proposal_embed_model, documents=documents_proposal, persist_dir="/workspace/data/compliance/vector_idx_proposed")
query_engine = vector_index.as_query_engine(
    similarity_top_k=5,
    # node_postprocessors=[
    #     LLMRerank(
    #         choice_batch_size=5,
    #         top_n=2,
    #     )
    # ],
    # see https://github.com/run-llama/llama_index/blob/f7c5ee5efbb6172e819f26d1705fcdf6114b11a3/llama-index-core/llama_index/core/response_synthesizers/type.py#L4
    response_mode="tree_summarize", # "accumulate", "compact_accumulate", "compact", "simple_summarize", "tree_summarize"
)

Settings.chunk_size = 500
Settings.chunk_overlap = 20
rules_vector_index = create_llama_vector_index_rag(llm, rules_embed_model, documents=documents, persist_dir="/workspace/data/compliance/vector_idx_sec")
graph_index = rules_vector_index.as_query_engine(
    similarity_top_k=5,
    # node_postprocessors=[
    #     LLMRerank(
    #         choice_batch_size=5,
    #         top_n=2,
    #     )
    # ],
    # see https://github.com/run-llama/llama_index/blob/f7c5ee5efbb6172e819f26d1705fcdf6114b11a3/llama-index-core/llama_index/core/response_synthesizers/type.py#L4
    response_mode="tree_summarize", # "accumulate", "compact_accumulate", "compact", "simple_summarize", "tree_summarize"
)

In [ ]:
response = query_engine.query("Summarize section IX.")
print(response)

## Summarize each section

In [8]:
import re
import ast

def parse_list_from_output_string(output_string):
    """
    Parses and extracts a Python list from the given output string.

    Args:
    output_string (str): The output string containing the list representation.

    Returns:
    list: The extracted Python list.
    """
    try:
        # Use regex to find the list portion within the string
        list_match = re.search(r'(\[[^\]]*\])', output_string, re.DOTALL)
        if list_match:
            list_str = list_match.group(1)
            # Safely evaluate the list string
            data = ast.literal_eval(list_str)
            if isinstance(data, list):
                return data
            else:
                raise ValueError("The extracted portion is not a list.")
        else:
            raise ValueError("No list found in the output string.")
    except (ValueError, SyntaxError):
        raise ValueError("The output string is not a valid list representation.")


In [ ]:
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.node_parser import SentenceSplitter

summarizer = TreeSummarize(llm=llm, verbose=True)
# original prompt
# prompt_summary = f"""
# Summarize this section of a new SEC rule proposal and/or amendment. 
# Make sure the summary includes every mention of any regulatory section, rule, or act (e.g. 12 U.S.C. 1843(k)(4)(C), 240.13a-15, Investment Company Act of 1940) in the text. 
# Do not use any prior knowledge except what is in the text to generate the summary.
# Be detailed and verbose in your response.
# Do not output any premable.
# """

# gpt enhanced prompt
prompt_summary = """
Summarize the content of the following section from a new SEC rule proposal or amendment. Ensure that the summary:

1. Includes every reference to any specific regulatory section, rule, or act mentioned in the text (e.g., 12 U.S.C. 1843(k)(4)(C), 240.13a-15, Investment Company Act of 1940).
2. Stays strictly within the information presented in the text, without incorporating any outside or prior knowledge.
3. Is detailed, thorough, and specific in its coverage of all key points.
4. Avoids adding any introductory or concluding remarks outside the scope of the summary itself.
"""

section_summaries = []
extracted_regs = []

for i,sec in enumerate(sections):
    rule_prompt = f"""
    Extract all mentions of any regulatory sections, rules, and acts in the following text.
    Compile all extracted items into a single Python List of Strings object.
    For example, ["Item 1", "Item 2", "Item 3"].

    Here is the text:
    {sec}
    """
    response = llm.complete(rule_prompt)

    # List checker
    rewrite_counter=0
    while True:
        try: 
            extracted_reg = parse_list_from_output_string(response.text)
            break
        except:
            print("Correcting list format...")
            response = llm.complete(f"""The following text does not contain a valid Python List of Strings? 
                                        Rewrite the text so that the List is in a valid Python format.
                                        For example, ["Item 1", "Item 2", "Item 3"].\n\n{response.text}
                                    """)
            rewrite_counter+=1
            print(f"   Rewrote list {rewrite_counter} times.")
            if rewrite_counter>5:
                raise ValueError("Could not correct list format.")

    extracted_regs.append(extracted_reg)
    prompt_summary = f"""
    Summarize the content of the following section from a new SEC rule proposal or amendment. Ensure that the summary:

    1. Includes every reference to any specific regulatory section, rule, or act mentioned in the text (e.g., 12 U.S.C. 1843(k)(4)(C), 240.13a-15, Unfunded Mandates Reform Act (section 202(a)), Investment Company Act of 1940).
    2. Stays strictly within the information presented in the text, without incorporating any outside or prior knowledge.
    3. Is detailed, thorough, and specific in its coverage of all key points.
    4. Avoids adding any introductory or concluding remarks outside the scope of the summary itself.

    Here are some of the regulatory sections, rules, and acts:
    {extracted_reg} 
    """
    print(f"Summarizing section {i+1}/{len(sections)}...")
    parser = SentenceSplitter(chunk_size=500, chunk_overlap=20)
    nodes = parser.get_nodes_from_documents([documents_proposal[i]], show_progress=True)
    response = await summarizer.aget_response(prompt_summary, [doc.text for doc in nodes])
    section_summaries.append(response)
    # if i==3:
    #     break


In [ ]:
for ex in extracted_regs:
    print(ex)
    print()

In [ ]:
goods=0
bads=0
for i,(sec,summary) in enumerate(zip(sections, section_summaries)):
    numbers = re.findall(r'(?<!\w)(\d{1,3}(?:,\d{3})*(?:\.\d+)?|\d+\.\d+|\d+)(?!\w)', summary)
    numbers = [x for x in numbers if len(x)>1]
    print(numbers)
    for number in numbers:
        if number not in sec:
            bads+=1
            print(f"{number} not in section {i}!")
        else:
            goods+=1

print(f"Goods: {goods}, Bads: {bads}")

In [ ]:
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Summary of Sections', 0, 1, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 14)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(5)

    def chapter_body(self, body):
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, body)
        self.ln()

def create_pdf(summaries, section_headers, output_filename):
    pdf = PDF()

    pdf.add_page()

    for header, summary in zip(section_headers, summaries):
        print(header)
        pdf.chapter_title(header.replace("’", "'"))
        pdf.chapter_body(summary.replace("–", "-"))

    pdf.output(output_filename)

# Save the summaries to a PDF
output_filename = "/workspace/data/summary_of_sections.pdf"
section_headers = [x.split('\n')[0].strip() for x in sections]
create_pdf(section_summaries, section_headers, output_filename)

In [ ]:
prompt_summary_of_summaries = f"""
Provide a high-level summary of the following section summaries from a new SEC rule proposal. Ensure that the summary:

1. Focuses on the most relevant points for a financial executive, emphasizing practical implications, compliance requirements, and potential business impacts.
2. Is thorough yet concise, presenting detailed insights that are actionable and strategic.
3. Avoids any preamble or extraneous details not directly related to the financial executive’s decision-making needs.
"""
summary_of_section_summaries = await summarizer.aget_response(prompt_summary_of_summaries, [doc for doc in section_summaries])

In [ ]:
print(summary_of_section_summaries)

In [ ]:
print(section_summaries[2])

In [ ]:
print(sections[1])

In [ ]:
# Securities Exchange Act of 1934
# Investment Advisers Act of 1940
# Gramm-Leach-Bliley Act
# 31 U.S.C. § 5311-5336
# 12 U.S.C. § 1843(k)(4)(C)
# 15 U.S.C. 80b (Investment Advisers Act of 1940)
# 15 U.S.C. 6809(2) (Gramm-Leach-Bliley Act)
# Securities Exchange Act of 1934
# 31 CFR part 1032
# 31 CFR 1023.220 (Customer Identification Program rule for broker-dealers)
# 31 CFR 1020.220 (CIP regulation)

## Agentic Version

In [1]:
from typing import List
import subprocess, os
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI as LOpenAI
from dotenv import load_dotenv
load_dotenv('/workspace/repos/agentic-ai/.env')

from utils import parse_list_from_output_string, extract_list_from_string
from llama_index.core.workflow import (
    Context,
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
)

from llama_index.core.agent.react import ReActAgent
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import ToolSelection, ToolOutput

from llama_index.core.chat_engine import SimpleChatEngine
from llama_index.utils.workflow import draw_all_possible_flows
from llama_index.core.response_synthesizers import TreeSummarize

from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI as LOpenAI
from rag_utils import create_llama_vector_index_rag

import os
import xml.etree.ElementTree as ET
from sec_utils import get_tree_data, get_metadata
from llamaindex_data_utils import extract_text_from_pdf

import os, re
from llama_index.readers.file import PDFReader
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter

from dotenv import load_dotenv
load_dotenv('/workspace/repos/agentic-ai/.env')

import nest_asyncio
nest_asyncio.apply()

from llama_index.embeddings.ollama import OllamaEmbedding

class SummaryStartEvent(Event):
    pass

# class InitializationEvent(Event):
#     pass

class InitializationCleanupEvent(Event):
    result: str

class RegulationsExtractionEvent(Event):
    pass

class FormatCorrectionEvent(Event):
    result: list

class SummarizationEvent(Event):
    result: list

class SummarizationNumericalValidationEvent(Event):
    result: list
    summaries: list


class RuleSummarizationFlow(Workflow):

    @step
    async def initialize(self, ctx: Context, ev: StartEvent) -> RegulationsExtractionEvent:
        
        # Open source work horse model
        model_name, ctx_len = "llama3.1:8b-instruct-q8_0", 128000
        addtion_kwargs = {"max_new_tokens": 8000}
        ctx.data["llm"] = Ollama(model=model_name, url="http://127.0.0.1:11434", context_window=ctx_len, model_type="chat", is_function_calling_model=True, 
                                    request_timeout=4000.0, **addtion_kwargs)
        
        # Expert closed model
        ctx.data["expert_llm"] = ctx.data["llm"] # OpenAI(model="gpt-4o",temperature=0.1)
        
        # embed_model = OllamaEmbedding(model_name, base_url="http://localhost:11434")
        
        parser = PDFReader(return_full_document=True)
        documents_proposal = parser.load_data("/workspace/data/compliance/FSI_Rule_proposal.pdf")

        # remove page numbers. they mess up later parsing.
        page_number = 1
        while True:
            # Create a regex pattern for the current page number
            pattern = rf'^{page_number}+ '
            # Substitute the pattern with an empty string
            new_text = re.sub(pattern, '', documents_proposal[0].text, flags=re.MULTILINE)
            # If no substitution was made, break the loop
            if new_text == documents_proposal[0].text:
                break
            # Update the text and increment the page number
            documents_proposal[0].text = new_text
            page_number += 1

        # TODO: Generalize (Very manual splitting of sections)
        pattern = r'(?<=\n)([IVX]+\..*?)(?=\n[IVX]+\.\s|\Z)'
        sections=[]
        sections.append("Proposal Information"+documents_proposal[0].text.split("I. Background A. Sta")[0])
        sections += re.findall(pattern, documents_proposal[0].text, re.DOTALL)
        section4_5split = sections[4].split("V. Paperwork Reduction Act")
        sections[4]=section4_5split[0]
        sections = sections[:5]+["V. Paperwork Reduction Act"+section4_5split[-1]]+sections[5:]
        for shead in [x.split('\n')[0].strip() for x in sections]:
            print(shead)


        documents_proposal = [Document(text=t, 
                                text_template='{metadata_str}\n\n{content}') for t in sections]

        # add metadata to the documents_proposal
        for i in range(len(documents_proposal)):
            documents_proposal[i].metadata["section"] = documents_proposal[i].text.split("\n")[0].strip()
            
        # documents_proposal.extend(docs)
        
        # Global state context
        ctx.data["num_sections"] = len(sections)
        ctx.data["bad_summaries"] = [x for x in range(len(sections))]
        ctx.data["chat"] = SimpleChatEngine.from_defaults(llm=ctx.data["expert_llm"])
        ctx.data["summarizer"] = TreeSummarize(llm=ctx.data["expert_llm"], verbose=False)
        ctx.data["new_rule_documents"] = documents_proposal
        
        ctx.data["initialized"] = True
        return RegulationsExtractionEvent()
    
    # @step
    # async def check_initialization(self, ctx: Context, ev: InitializationEvent) -> InitializationCleanupEvent:
    #     assert ctx.data["initialized"], "Workflow not initialized."
    #     for doc in ctx.data["new_rule_documents"]:
    #         response = await ctx.data["chat"].chat(f"""
    #         Given text, determine if there are any formatting issues. 
    #         If it's good, return the original text.
    #         If it's bad, rewrite the text so that it the text is clean.
    #         Bad formatting contains accidental spaces (e.g. "Co mputer", "tit- for-tat") and .

    #         Here is the text: {doc.text}
    #         """)
        
    #         print("Query Judge response:", response)
    #         if response == "bad":
    #             # try again
    #             return BadQueryEvent(query=ev.query)
    #         else:
    #             pass

    #     return SummaryStartEvent()

    @step
    async def extract_regulations(self, ctx: Context, ev: RegulationsExtractionEvent) -> FormatCorrectionEvent:
        assert ctx.data["initialized"], "Workflow not initialized."
        
        regs_extraction=[]
        for i,sec in enumerate(ctx.data["new_rule_documents"]):

            rule_prompt = f"""
            Extract all mentions of any regulatory sections, rules, and acts in the following text.
            Compile all extracted items into a single Python List of Strings object.
            For example, ["Item 1", "Item 2", "Item 3"].
            Only return the Python List of Strings.

            Here is the text:
            {sec}
            """
            response = ctx.data["chat"].chat(rule_prompt)
            regs_extraction.append(response)
        
        return FormatCorrectionEvent(result=regs_extraction)
    
    @step
    async def correct_format(self, ctx: Context, ev: FormatCorrectionEvent) -> SummarizationEvent:
        
        regs_list = []
        for response in ev.result:
            # List checker
            rewrite_counter=0
            while True:
                try: 
                    
                    # extracted_reg = parse_list_from_output_string(response)
                    extracted_reg = extract_list_from_string(response)
                    break
                except:
                    print("Correcting list format...")
                    response = ctx.data["chat"].chat(f"""The following text does not contain a valid Python List of Strings? 
                                                Rewrite the text so that the List is in a valid Python format.
                                                For example, ["Item 1", "Item 2", "Item 3"].\nText:\n\n{response}
                                            """)
                    rewrite_counter+=1
                    print(f"   Rewrote list {rewrite_counter} times.")
                    if rewrite_counter>5:
                        raise ValueError("Could not correct list format.")
            regs_list.append(extracted_reg)

        return SummarizationEvent(result=regs_list)


    @step
    async def summarize_sections(self, ctx: Context, ev: SummarizationEvent) -> SummarizationNumericalValidationEvent | StopEvent: #SummarizationValidationEvent:
        
        if len(ctx.data["bad_summaries"])==0:
            return StopEvent(result=ctx.data["section_summaries"])
        elif "section_summaries" in ctx.data:
            print("   Bad summaries found. Correcting...")
            section_summaries = ctx.data["section_summaries"]
            summary_count = ctx.data["bad_summaries"]
            prompt_suffix = f"""\nThe first attempt at summarizing this section had numerical copy mistakes. Copy numbers exactly as they appear in the text."""
        else:
            section_summaries = [None]*ctx.data["num_sections"]
            summary_count = range(ctx.data["num_sections"])
            prompt_suffix = ""
        
        for i in summary_count:
            extracted_reg = ev.result[i]
            prompt_summary = f"""
            Summarize the content of the following section from a new SEC rule proposal or amendment. Ensure that the summary:

            1. Includes every reference to any specific regulatory section, rule, or act mentioned in the text (e.g., 12 U.S.C. 1843(k)(4)(C), 240.13a-15, Unfunded Mandates Reform Act (section 202(a)), Investment Company Act of 1940).
            2. Stays strictly within the information presented in the text, without incorporating any outside or prior knowledge.
            3. Is detailed, thorough, and specific in its coverage of all key points, definitions and exclusions.
            4. Avoids adding any introductory or concluding remarks outside the scope of the summary itself.

            Here are some of the regulatory sections, rules, and acts:
            {extracted_reg} 
            """
            print(f"   Summarizing section {i+1}/{ctx.data['num_sections']}...")
            parser = SentenceSplitter(chunk_size=500, chunk_overlap=20)
            nodes = parser.get_nodes_from_documents(ctx.data["new_rule_documents"], show_progress=True)
            response = ctx.data["summarizer"].get_response(prompt_summary+prompt_suffix, [doc.text for doc in nodes])
            section_summaries[i] = response
        
        ctx.data["section_summaries"] = section_summaries
        return SummarizationNumericalValidationEvent(result=ev.result, summaries=section_summaries)
    
    @step
    async def validate_summaries(self, ctx: Context, ev: SummarizationNumericalValidationEvent) -> SummarizationEvent:
        
        bad_summaries = []
        sections = [x.text for x in ctx.data["new_rule_documents"]]
        for i in ctx.data["bad_summaries"]:
            goods=0
            bads=0
            numbers = re.findall(r'(?<!\w)(\d{1,3}(?:,\d{3})*(?:\.\d+)?|\d+\.\d+|\d+)(?!\w)', ev.summaries[i])
            numbers = [x for x in numbers if len(x)>1]
            for number in numbers:
                if number not in sections[i]:
                    bads+=1
                    # print(f"{number} not in section {i}!")
                else:
                    goods+=1
        
            # TODO: this is a manual param. put in ctx.data
            if bads>7:
                bad_summaries.append(i)
            print(f"   Section {i} Goods: {goods}, Bads: {bads}")

        ctx.data["bad_summaries"] = bad_summaries            
        return SummarizationEvent(result=ev.result)


In [ ]:
draw_all_possible_flows(RuleSummarizationFlow,filename="rule_flows.html")

In [ ]:
c = RuleSummarizationFlow(timeout=12000, verbose=True)
result = await c.run()

In [ ]:
result

In [ ]:
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Summary of Sections', 0, 1, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 14)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(5)

    def chapter_body(self, body):
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, body)
        self.ln()

def create_pdf(summaries, section_headers, output_filename):
    pdf = PDF()

    pdf.add_page()

    for header, summary in zip(section_headers, summaries):
        print(header)
        pdf.chapter_title(header.replace("’", "'"))
        # try:
        pdf.chapter_body(summary.replace("“", '"').replace("”", '"').replace("’", "-").replace("–", "-").replace("—", "-"))
        # except:
            # print(summary)
        # pdf.chapter_body(summary.)
        # pdf.chapter_body(summary)
        

    pdf.output(output_filename)

# Save the summaries to a PDF
output_filename = "/workspace/data/summary_of_sections.pdf"
# section_headers = [x.split('\n')[0].strip() for x in sections]
section_headers = ["Proposal Information",
"I. Background A. Statutory Provisions",
"II. Section -by-Section Analysis",
"III. Request for Comments",
"IV. Analysis of the Costs and Benefits Associated with the Proposed Rule",
"V. Paperwork Reduction Act",
"VI. Regulatory Flexibility Act",
"VII. Considerations of th e Impact on the Economy",
"IX. FinCEN’s Unfunded Mandates Reform Act Determination"
]
create_pdf(result, section_headers, output_filename)

## Agentic First Version

In [2]:
from llama_index.core.agent.react import ReActAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import ToolSelection, ToolOutput
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    Settings,
    Document,
    load_index_from_storage
)
from llama_index.core.workflow import (
    step,
    Context,
    Workflow,
    Event,
    StartEvent,
    StopEvent
)
from llama_index.core.postprocessor.rankGPT_rerank import RankGPTRerank
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.chat_engine import SimpleChatEngine
from llama_index.utils.workflow import draw_all_possible_flows
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.response_synthesizers import TreeSummarize

from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI as LOpenAI
from rag_utils import create_llama_vector_index_rag

import os
import xml.etree.ElementTree as ET
from sec_utils import get_tree_data, get_metadata
from llamaindex_data_utils import extract_text_from_pdf

from dotenv import load_dotenv
load_dotenv('/workspace/repos/agentic-ai/.env')



In [ ]:
import os, re
from llama_index.readers.file import PDFReader
from llamaindex_data_utils import extract_text_from_pdf
from llama_index.core import Document
from dotenv import load_dotenv
load_dotenv('/workspace/repos/agentic-ai/.env')

access_token = os.getenv('HF_TOKEN')
llama_api_key = os.getenv('LLAMA_API_KEY')

parser = PDFReader(return_full_document=True)
documents_proposal = parser.load_data("/workspace/data/compliance/FSI_Rule_proposal.pdf")
# Regular expression to find numbers followed by a space at the beginning of a new line
pattern = r'^\d+ '
# Replace page number pattern with an empty string
documents_proposal[0].text = re.sub(pattern, '', documents_proposal[0].text, flags=re.MULTILINE)

pattern = r'(?<=\n)([IVX]+\..*?)(?=\n[IVX]+\.\s|\Z)'
sections=[]
sections.append("Proposal Information"+documents_proposal[0].text.split("I. Background A. Sta")[0])
sections += re.findall(pattern, documents_proposal[0].text, re.DOTALL)
section4_5split = sections[4].split("V. Paperwork Reduction Act")
sections[4]=section4_5split[0]
sections.append("V. Paperwork Reduction Act"+section4_5split[-1])

# pdf_urls = ["/workspace/data/compliance/FSI_Factsheet.pdf", "/workspace/data/compliance/FSI_Press_Release.pdf", "/workspace/data/compliance/FSI_Rule_proposal.pdf"]
# descriptions = ["Factsheet about newly proposed SEC rule.", "Press release regarding newly proposed SEC rule.", "The full text of the newly proposed SEC rule."]
# pdf_urls = ["/workspace/data/compliance/FSI_Rule_proposal.pdf"]
# descriptions = ["The full text of the newly proposed SEC rule."]
# documents_proposal = extract_text_from_pdf(pdf_urls, llama_api_key, llamaparse_kwargs={"split_by_page":False}, save_json_path=None)
documents_proposal = [Document(text=t, 
                          text_template='{metadata_str}\n\n{content}') for t in sections]

# add metadata to the documents_proposal
for i in range(len(documents_proposal)):
    documents_proposal[i].metadata["section"] = documents_proposal[i].text.split("\n")[0].strip()
    # documents_proposal[i].metadata["source"] = pdf_urls[i]
    # documents_proposal[i].metadata["description"] = descriptions[i]
    
# documents_proposal.extend(docs)

In [6]:
#### Define the workflow and all events
# class InputEvent(Event):
#     input: list[ChatMessage]

# class ToolCallEvent(Event):
#     tool_calls: list[ToolSelection]

# class FunctionOutputEvent(Event):
#     output: ToolOutput

class InitializeEvent(Event):
    pass

class QueryEvalEvent(Event):
    query: str

class BadQueryEvent(Event):
    query: str

class NaiveRAGEvent(Event):
    query: str

class HighTopKEvent(Event):
    query: str

class RerankEvent(Event):
    query: str

class ComparisonEvent(Event):
    query: str
    documents: str

class ResponseEvent(Event):
    query: str
    response: str

class SummarizeEvent(Event):
    query: str
    documents: str

In [7]:
import nest_asyncio
nest_asyncio.apply()

class ComplianceWorkflow(Workflow):

    @step(pass_context=True)
    async def initialize(self, ctx: Context, ev: InitializeEvent) -> QueryEvalEvent:
        
        # Open source work horse model
        model_name, ctx_len = "llama3.1:latest", 128000
        addtion_kwargs = {"max_new_tokens": 4000}
        # system_prompt = "You are an expert at answering questions about rules and regulations regarding Title 17—Commodity and Securities Exchanges: CHAPTER II—SECURITIES AND EXCHANGE COMMISSION. Please provide a summary of the following text, and cite any sections, rules, acts or laws (e.g. § 230.503, § 240.13a-15, Act (15 U.S.C. 781), Investment Company Act of 1940) from context that support the answer. Be detailed in your response."
        ctx.data["llm"] = Ollama(model=model_name, url="http://127.0.0.1:11434", context_window=ctx_len, model_type="chat", is_function_calling_model=True, 
                                    request_timeout=4000.0, additional_kwargs=addtion_kwargs) #, system_prompt=system_prompt)
        
        # Expert closed model
        ctx.data["expert_llm"] = ctx.data["llm"] # OpenAI(model="gpt-4o",temperature=0.1)
        
        # Settings.llm = ctx.data["llm"]
        # Embedding models
        embed_model_name = "dunzhang/stella_en_1.5B_v5" 
        # finetuned_embed_model_name = "/workspace/data/compliance/rule_proposal_embedding_model_gpt"
        # proposal_embed_model = HuggingFaceEmbedding(model_name=finetuned_embed_model_name)
        sec_embed_model = HuggingFaceEmbedding(model_name=embed_model_name)
        
        # ctx.data["new_rule_index"] = create_llama_vector_index_rag(llm=ctx.data["llm"], 
        #                                                             embed_model=proposal_embed_model, 
        #                                                             persist_dir="/workspace/data/compliance/vector_idx_proposed",
        #                                                             vector_store_kwargs={"chunk_size": 300, "chunk_overlap": 20})
        
        ctx.data["sec_index"] = create_llama_vector_index_rag(llm=ctx.data["llm"], 
                                                                embed_model=sec_embed_model, 
                                                                persist_dir="/workspace/data/compliance/vector_idx_sec",
                                                                vector_store_kwargs={"chunk_size": 500, "chunk_overlap": 20})
        
        # we use a chat engine so it remembers previous interactions
        ctx.data["judge"] = SimpleChatEngine.from_defaults(llm=ctx.data["expert_llm"])
        ctx.data["summarizer"] = TreeSummarize(llm=ctx.data["expert_llm"], verbose=False)
        ctx.data["new_rule_documents"] = documents_proposal

        ctx.data["initialized"] = True
        return QueryEvalEvent()

    @step(pass_context=True)
    async def judge_query(self, ctx: Context, ev: StartEvent | QueryEvalEvent ) -> BadQueryEvent | RerankEvent | SummarizeEvent: #NaiveRAGEvent | HighTopKEvent |
        # TODO: make initialize its own event
        # initialize
        if 'initialized' not in ctx.data:
            return InitializeEvent()

        response = await ctx.data["judge"].chat(f"""
            Given a user query, determine if this is likely to yield precise results from a financial RAG system as-is. 
            If it's good, return 'good', if it's bad, return 'bad'.
            Good queries use a lot of relevant keywords and are detailed. Bad queries are vague or ambiguous.

            Here is the query: {ev.query}
            """)
        
        print("Query Judge response:", response)
        if response == "bad":
            # try again
            return BadQueryEvent(query=ev.query)
        else:
            # send query to all 3 strategies
            # self.send_event(NaiveRAGEvent(query=ev.query))
            # self.send_event(HighTopKEvent(query=ev.query))
            await self.send_event(RerankEvent(query=ev.query))
            await self.send_event(SummarizeEvent(query=ev.query, documents=ctx.data["new_rule_documents"]))

    @step(pass_context=True)
    async def improve_query(self, ctx: Context, ev: BadQueryEvent) -> QueryEvalEvent:
        response = await ctx.data["llm"].complete(f"""
            This is a query to a RAG system: {ev.query}

            The query is bad because it is too vague. Please provide a more detailed query that includes specific keywords and removes any ambiguity.
        """)
        return QueryEvalEvent(query=str(response))

    @step(pass_context=True)
    async def naive_rag(self, ctx: Context, ev: NaiveRAGEvent) -> ResponseEvent:
        index = ctx.data["new_rule_index"]
        engine = index.as_query_engine(llm=ctx.data["expert_llm"], similarity_top_k=3)
        response = await engine.query(ev.query)
        print("Naive response:", response)
        return ResponseEvent(query=ev.query, source="Naive", response=str(response))

    @step(pass_context=True)
    async def high_top_k(self, ctx: Context, ev: HighTopKEvent) -> ResponseEvent:
        index = ctx.data["new_rule_index"]
        engine = index.as_query_engine(llm=ctx.data["expert_llm"], similarity_top_k=5)
        response = await engine.query(ev.query)
        print("High top k response:", response)
        return ResponseEvent(query=ev.query, source="High top k", response=str(response))

    @step(pass_context=True)
    async def extract_sec_sections(self, ctx: Context, ev: RerankEvent) -> ResponseEvent:
        pass

    @step(pass_context=True)
    async def rerank(self, ctx: Context, ev: RerankEvent) -> ResponseEvent:
        index = ctx.data["new_rule_index"]
        reranker = RankGPTRerank(
            top_n=2,
            llm=ctx.data["expert_llm"]
        )
        retriever = index.as_retriever(llm=ctx.data["expert_llm"], similarity_top_k=5)
        engine = RetrieverQueryEngine.from_args(
            retriever=retriever,
            node_postprocessors=[reranker],
        )
        response = await engine.query(ev.query)
        print("Reranker response:", response)
        return ResponseEvent(query=ev.query, source="Reranker", response=str(response))

    @step(pass_context=True)
    async def summarize(self, ctx: Context, ev: SummarizeEvent) -> ResponseEvent:
        prompt_summary = f"""
        A long document of a newly proposed SEC rule has been provided. 
        Please provide a summary of the document with details on key points and sections
        that would be relevant to a compliance officer.
        """

        response = await ctx.data["summarizer"].aget_response(prompt_summary, [doc.text for doc in ev.documents])
        print("Summarizer response:", response)
        return ResponseEvent(query=ev.query, source="Summary", response=str(response))

    @step(pass_context=True)
    async def judge(self, ctx: Context, ev: ResponseEvent) -> StopEvent:
        ready = ctx.collect_events(ev, [ResponseEvent]*2)
        if ready is None:
            return None

        response = ctx.data["judge"].chat(f"""
            A user has provided a query and 3 different strategies have been used
            to try to answer the query. Your job is to decide which strategy best
            answered the query. The query was: {ev.query}

            Response 1 ({ready[0].source}): {ready[0].response}
            Response 2 ({ready[1].source}): {ready[1].response}
            Response 3 ({ready[2].source}): {ready[2].response}

            Please provide the number of the best response (1, 2, or 3).
            Just provide the number, with no other text or preamble.
        """)

        summary_idx = [i for i, r in enumerate(ready) if r.source == "Summary"][0]
        best_response = int(str(response))
        print(f"Best response was number {best_response}, which was from {ready[best_response-1].source}")
        return StopEvent(result=[str(ready[best_response-1].response), ready[summary_idx].response])
        # return StopEvent(result=str(ready[best_response-1].response))

In [ ]:
draw_all_possible_flows(ComplianceWorkflow,filename="concierge_flows.html")

In [ ]:
c = ComplianceWorkflow(timeout=120, verbose=True)
result = await c.run(
    query="What SEC sections does the newly proposed rule affect?"
)
print(result)

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(chunk_size=500, chunk_overlap=50)
nodes = parser.get_nodes_from_documents(documents_proposal, show_progress=True)
len(nodes)

In [ ]:
from llama_index.core.response_synthesizers import TreeSummarize
summarizer = TreeSummarize(llm=llm, verbose=True)
# prompt_summary = "You are a professional executive of AlphaTrAI. Your job is to summarize this text in great detail from a video transcription. The summary will be distributed to investors and stakeholders, so give a lot of details and examples from the transcription."
prompt_summary = """
You are an expert compliance officer who specializes at examining multiple documents for the purpose of extracting novelty, importance, and summarizing findings. Your task is to read the following documents and provide a summary of the key points, sections, and rules that are relevant to a compliance officer. Be detailed in your response.
"""

response = await summarizer.aget_response(prompt_summary, [doc.text for doc in nodes])

In [ ]:
from llama_index.core.agent.function_calling 
response_information_tool = []
llm.chat_with_tools()

In [ ]:
#### Define the workflow and all events
# class InputEvent(Event):
#     input: list[ChatMessage]

# class ToolCallEvent(Event):
#     tool_calls: list[ToolSelection]

# class FunctionOutputEvent(Event):
#     output: ToolOutput

class InitializeEvent(Event):
    pass

class JudgeEvent(Event):
    query: str

class BadQueryEvent(Event):
    query: str

class NaiveRAGEvent(Event):
    query: str

class HighTopKEvent(Event):
    query: str

class RerankEvent(Event):
    query: str

class ComparisonEvent(Event):
    query: str
    documents: str

class ResponseEvent(Event):
    query: str
    response: str

class SummarizeEvent(Event):
    query: str
    documents: str

In [ ]:
class ComplianceWorkflow(Workflow):

    def load_or_create_index(self, persist_dir, documents=None):
        # Check if the index already exists
        if os.path.exists(persist_dir):
            print("Loading existing index...")
            # Load the index from disk
            storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
            index = load_index_from_storage(storage_context)
        else:
            print("Creating new index...")
            # Load documents from the specified directory
            # TODO: add a check for the documents

            # Create a new index from the documents
            index = VectorStoreIndex.from_documents(documents)

            # Persist the index to disk
            index.storage_context.persist(persist_dir=persist_dir)

        return index
    
    @step(pass_context=True)
    async def initialize(self, ctx: Context, ev: InitializeEvent) -> JudgeEvent:
        
        # Open source work horse model
        model_name, ctx_len = "hermes3:8b", 128000
        addtion_kwargs = {"max_new_tokens": 2000}
        # system_prompt = "You are an expert at answering questions about rules and regulations regarding Title 17—Commodity and Securities Exchanges: CHAPTER II—SECURITIES AND EXCHANGE COMMISSION. Please provide a summary of the following text, and cite any sections, rules, acts or laws (e.g. § 230.503, § 240.13a-15, Act (15 U.S.C. 781), Investment Company Act of 1940) from context that support the answer. Be detailed in your response."
        ctx.data["llm"] = Ollama(model=model_name, url="http://127.0.0.1:11434", context_window=ctx_len, model_type="chat", is_function_calling_model=True, 
                                    request_timeout=4000.0, additional_kwargs=addtion_kwargs) #, system_prompt=system_prompt)
        
        # Expert closed model
        ctx.data["expert_llm"] = ctx.data["llm"] # OpenAI(model="gpt-4o",temperature=0.1)
        
        # Settings.llm = ctx.data["llm"]
        # Embedding models
        embed_model_name = "dunzhang/stella_en_1.5B_v5" 
        finetuned_embed_model_name = "/workspace/data/compliance/rule_proposal_embedding_model_gpt"
        proposal_embed_model = HuggingFaceEmbedding(model_name=finetuned_embed_model_name)
        sec_embed_model = HuggingFaceEmbedding(model_name=embed_model_name)
        
        ctx.data["new_rule_index"] = create_llama_vector_index_rag(llm=ctx.data["llm"], 
                                                                    embed_model=proposal_embed_model, 
                                                                    persist_dir="/workspace/data/compliance/vector_idx_proposed",
                                                                    vector_store_kwargs={"chunk_size": 300, "chunk_overlap": 20})
        
        ctx.data["sec_index"] = create_llama_vector_index_rag(llm=ctx.data["llm"], 
                                                                embed_model=sec_embed_model, 
                                                                persist_dir="/workspace/data/compliance/vector_idx_sec",
                                                                vector_store_kwargs={"chunk_size": 500, "chunk_overlap": 20})
        
        # we use a chat engine so it remembers previous interactions
        ctx.data["judge"] = SimpleChatEngine.from_defaults(llm=ctx.data["expert_llm"])
        ctx.data["summarizer"] = TreeSummarize(llm=ctx.data["expert_llm"], verbose=False)
        ctx.data["new_rule_documents"] = documents_proposal

        ctx.data["initialized"] = True
        return JudgeEvent()

    @step(pass_context=True)
    async def judge_query(self, ctx: Context, ev: StartEvent | JudgeEvent ) -> BadQueryEvent | NaiveRAGEvent | HighTopKEvent | RerankEvent | SummarizeEvent:
        # TODO: make initialize its own event
        # initialize
        if 'initialized' not in ctx.data:
            return InitializeEvent()

        response = ctx.data["judge"].chat(f"""
            Given a user query, determine if this is likely to yield good results from a RAG system as-is. If it's good, return 'good', if it's bad, return 'bad'.
            Good queries use a lot of relevant keywords and are detailed. Bad queries are vague or ambiguous.

            Here is the query: {ev.query}
            """)
        if response == "bad":
            # try again
            return BadQueryEvent(query=ev.query)
        else:
            # send query to all 3 strategies
            self.send_event(NaiveRAGEvent(query=ev.query))
            self.send_event(HighTopKEvent(query=ev.query))
            self.send_event(RerankEvent(query=ev.query))
            self.send_event(SummarizeEvent(query=ev.query, documents=ctx.data["new_rule_documents"]))

    @step(pass_context=True)
    async def improve_query(self, ctx: Context, ev: BadQueryEvent) -> JudgeEvent:
        response = ctx.data["llm"].complete(f"""
            This is a query to a RAG system: {ev.query}

            The query is bad because it is too vague. Please provide a more detailed query that includes specific keywords and removes any ambiguity.
        """)
        return JudgeEvent(query=str(response))

    @step(pass_context=True)
    async def naive_rag(self, ctx: Context, ev: NaiveRAGEvent) -> ResponseEvent:
        index = ctx.data["new_rule_index"]
        engine = index.as_query_engine(llm=ctx.data["expert_llm"], similarity_top_k=3)
        response = engine.query(ev.query)
        print("Naive response:", response)
        return ResponseEvent(query=ev.query, source="Naive", response=str(response))

    @step(pass_context=True)
    async def high_top_k(self, ctx: Context, ev: HighTopKEvent) -> ResponseEvent:
        index = ctx.data["new_rule_index"]
        engine = index.as_query_engine(llm=ctx.data["expert_llm"], similarity_top_k=5)
        response = engine.query(ev.query)
        print("High top k response:", response)
        return ResponseEvent(query=ev.query, source="High top k", response=str(response))

    @step(pass_context=True)
    async def rerank(self, ctx: Context, ev: RerankEvent) -> ResponseEvent:
        index = ctx.data["new_rule_index"]
        reranker = RankGPTRerank(
            top_n=2,
            llm=ctx.data["expert_llm"]
        )
        retriever = index.as_retriever(llm=ctx.data["expert_llm"], similarity_top_k=5)
        engine = RetrieverQueryEngine.from_args(
            retriever=retriever,
            node_postprocessors=[reranker],
        )
        response = engine.query(ev.query)
        print("Reranker response:", response)
        return ResponseEvent(query=ev.query, source="Reranker", response=str(response))

    @step(pass_context=True)
    async def summarize(self, ctx: Context, ev: SummarizeEvent) -> ResponseEvent:
        prompt_summary = f"""
        A long document of a newly proposed SEC rule has been provided. 
        Please provide a summary of the document with details on key points and sections
        that would be relevant to a compliance officer.
        """

        response = await ctx.data["summarizer"].aget_response(prompt_summary, [doc.text for doc in ev.documents])
        print("Summarizer response:", response)
        return ResponseEvent(query=ev.query, source="Summary", response=str(response))


    @step(pass_context=True)
    async def judge(self, ctx: Context, ev: ResponseEvent) -> StopEvent:
        ready = ctx.collect_events(ev, [ResponseEvent]*4)
        if ready is None:
            return None

        response = ctx.data["judge"].chat(f"""
            A user has provided a query and 3 different strategies have been used
            to try to answer the query. Your job is to decide which strategy best
            answered the query. The query was: {ev.query}

            Response 1 ({ready[0].source}): {ready[0].response}
            Response 2 ({ready[1].source}): {ready[1].response}
            Response 3 ({ready[2].source}): {ready[2].response}

            Please provide the number of the best response (1, 2, or 3).
            Just provide the number, with no other text or preamble.
        """)

        summary_idx = [i for i, r in enumerate(ready) if r.source == "Summary"][0]
        best_response = int(str(response))
        print(f"Best response was number {best_response}, which was from {ready[best_response-1].source}")
        return StopEvent(result=[str(ready[best_response-1].response), ready[-1].response])
        # return StopEvent(result=str(ready[best_response-1].response))

In [ ]:

graph_engine_tools = QueryEngineTool(
            query_engine=graph_index,
            metadata=ToolMetadata(
                name="sec_title_17_chapter_ii_tool",
                description=(
                    "Contains all the current sections, rules, and relationships of SEC Title 17 Chapter II."
                ),
            ),
        )

query_engine_tools = QueryEngineTool(
            query_engine=query_engine,
            metadata=ToolMetadata(
                name="new_rule_proposal_tool",
                description=(
                    "Contains all the information about the newly proposed SEC rule."
                ),
            ),
        )


In [13]:
from typing import Any, List

from llama_index.core.llms.function_calling import FunctionCallingLLM
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.tools.types import BaseTool
from llama_index.core.workflow import Workflow, StartEvent, StopEvent, step


class FuncationCallingAgent(Workflow):
    def __init__(
        self,
        *args: Any,
        llm: FunctionCallingLLM | None = None,
        tools: List[BaseTool] | None = None,
        **kwargs: Any,
    ) -> None:
        super().__init__(*args, **kwargs)
        self.tools = tools or []

        self.llm = llm
        assert self.llm.metadata.is_function_calling_model

        self.memory = ChatMemoryBuffer.from_defaults(llm=llm)
        self.sources = []

    @step
    async def prepare_chat_history(self, ev: StartEvent) -> InputEvent:
        # clear sources
        self.sources = []

        # get user input
        user_input = ev.input
        user_msg = ChatMessage(role="user", content=user_input)
        self.memory.put(user_msg)

        # get chat history
        chat_history = self.memory.get()
        return InputEvent(input=chat_history)

    @step
    async def handle_llm_input(
        self, ev: InputEvent
    ) -> ToolCallEvent | StopEvent:
        chat_history = ev.input
        '''
        Takes in the chat history, and uses tools to generate a response.
        '''
        response = await self.llm.achat_with_tools(
            self.tools, chat_history=chat_history
        )
        self.memory.put(response.message)

        tool_calls = self.llm.get_tool_calls_from_response(
            response, error_on_no_tool_call=False
        )

        if not tool_calls:
            return StopEvent(
                result={"response": response, "sources": [*self.sources]}
            )
        else:
            return ToolCallEvent(tool_calls=tool_calls)

    @step
    async def handle_tool_calls(self, ev: ToolCallEvent) -> InputEvent:
        tool_calls = ev.tool_calls
        tools_by_name = {tool.metadata.get_name(): tool for tool in self.tools}

        tool_msgs = []

        # call tools -- safely!
        for tool_call in tool_calls:
            tool = tools_by_name.get(tool_call.tool_name)
            additional_kwargs = {
                "tool_call_id": tool_call.tool_id,
                "name": tool.metadata.get_name(),
            }
            if not tool:
                tool_msgs.append(
                    ChatMessage(
                        role="tool",
                        content=f"Tool {tool_call.tool_name} does not exist",
                        additional_kwargs=additional_kwargs,
                    )
                )
                continue

            try:
                tool_output = tool(**tool_call.tool_kwargs)
                self.sources.append(tool_output)
                tool_msgs.append(
                    ChatMessage(
                        role="tool",
                        content=tool_output.content,
                        additional_kwargs=additional_kwargs,
                    )
                )
            except Exception as e:
                tool_msgs.append(
                    ChatMessage(
                        role="tool",
                        content=f"Encountered error in tool call: {e}",
                        additional_kwargs=additional_kwargs,
                    )
                )

        for msg in tool_msgs:
            self.memory.put(msg)

        chat_history = self.memory.get()
        return InputEvent(input=chat_history)

In [ ]:
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI

llm.is_function_calling_model = True

agent = FuncationCallingAgent(
    llm=llm, tools=all_tools, timeout=120, verbose=True
)

ret = await agent.run(input="What is a summary of the proposed rule, and what SEC rules does it change?")

In [ ]:
print(ret["response"])


In [ ]:
from llama_index.utils.workflow import draw_all_possible_flows
from llama_index.utils.workflow import draw_most_recent_execution
from IPython.display import display, HTML
draw_all_possible_flows(FuncationCallingAgent, "first_func_agent.html")

In [ ]:
from IPython.display import IFrame

IFrame(src='/workspace/repos/agentic-ai/first_func_agent.html', width=700, height=600)

In [ ]:
from IPython.display import display, HTML

# Read the HTML file content
with open('/workspace/repos/agentic-ai/first_func_agent.html', 'r') as file:
    html_content = file.read()

# Render the HTML content in a Jupyter cell
display(HTML(html_content))

In [5]:
from typing import Optional
import asyncio
from pydantic_settings import BaseSettings
import signal
from llama_deploy.deploy.deploy import (
                        _get_message_queue_client,
                        _deploy_local_message_queue,
                        _get_shutdown_handler
                    )

from llama_deploy import (
    deploy_core,
    ControlPlaneConfig,
    SimpleMessageQueueConfig,
    SimpleOrchestratorConfig,
    ControlPlaneServer,
    SimpleOrchestrator,
    LlamaDeployClient
)


In [ ]:
from llama_deploy import (
    deploy_core,
    ControlPlaneConfig,
    SimpleMessageQueueConfig,
)


async def main():
    await deploy_core(
        control_plane_config=ControlPlaneConfig(port=8002),
        message_queue_config=SimpleMessageQueueConfig(port=8003),
    )


# if __name__ == "__main__":
#     import asyncio

#     asyncio.run(main())

In [ ]:
from llama_deploy import (
    deploy_workflow,
    WorkflowServiceConfig,
    ControlPlaneConfig,
    SimpleMessageQueueConfig,
)
from llama_index.core.workflow import Workflow, StartEvent, StopEvent, step


# create a dummy workflow
class MyWorkflow(Workflow):
    @step()
    async def run_step(self, ev: StartEvent) -> StopEvent:
        # Your workflow logic here
        arg1 = str(ev.get("arg1", ""))
        result = arg1 + "_result"
        return StopEvent(result=result)


async def main():
    await deploy_workflow(
        workflow=MyWorkflow(),
        workflow_config=WorkflowServiceConfig(
            host="127.0.0.1", port=8004, service_name="my_workflow"
        ),
        control_plane_config=ControlPlaneConfig(),
    )


if __name__ == "__main__":
    import asyncio

    asyncio.run(main())

In [ ]:
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Context,
)
import random
from llama_index.core.workflow import draw_all_possible_flows
from llama_index.utils.workflow import draw_most_recent_execution

In [22]:
from typing import Optional
import asyncio
from pydantic_settings import BaseSettings
import signal
from llama_deploy.deploy.deploy import (
                        _get_message_queue_client,
                        _deploy_local_message_queue,
                        _get_shutdown_handler
                    )

from llama_deploy import (
    deploy_core,
    ControlPlaneConfig,
    SimpleMessageQueueConfig,
    SimpleOrchestratorConfig,
    ControlPlaneServer,
    SimpleOrchestrator,
    LlamaDeployClient
)


async def deploy_core(
    control_plane_config: ControlPlaneConfig,
    message_queue_config: BaseSettings,
    orchestrator_config: Optional[SimpleOrchestratorConfig] = None,
) -> None:
    orchestrator_config = orchestrator_config or SimpleOrchestratorConfig()

    message_queue_client = _get_message_queue_client(message_queue_config)

    control_plane = ControlPlaneServer(
        message_queue_client,
        SimpleOrchestrator(**orchestrator_config.model_dump()),
        **control_plane_config.model_dump(),
    )

    message_queue_task = None
    if isinstance(message_queue_config, SimpleMessageQueueConfig):
        message_queue_task = _deploy_local_message_queue(message_queue_config)

    control_plane_task = asyncio.create_task(control_plane.launch_server())

    # let services spin up
    await asyncio.sleep(1)

    # register the control plane as a consumer
    control_plane_consumer_fn = await control_plane.register_to_message_queue()

    consumer_task = asyncio.create_task(control_plane_consumer_fn())

    # let things sync up
    await asyncio.sleep(1)

    # let things run
    if message_queue_task:
        all_tasks = [control_plane_task, consumer_task, message_queue_task]
    else:
        all_tasks = [control_plane_task, consumer_task]

    shutdown_handler = _get_shutdown_handler(all_tasks)
    loop = asyncio.get_event_loop()
    while loop.is_running():
        await asyncio.sleep(0.1)
        signal.signal(signal.SIGINT, shutdown_handler)

        for task in all_tasks:
            if task.done() and task.exception():  # type: ignore
                raise task.exception()  # type: ignore

In [24]:
import httpx
from llama_index.core.workflow import Workflow
from llama_deploy import (
    WorkflowServiceConfig,
    WorkflowService,
)

from llama_deploy.deploy.deploy import (
                        _get_message_queue_config,
                    )

async def deploy_workflow(
    workflow: Workflow,
    workflow_config: WorkflowServiceConfig,
    control_plane_config: ControlPlaneConfig,
) -> None:
    control_plane_url = control_plane_config.url

    async with httpx.AsyncClient() as client:
        response = await client.get(f"{control_plane_url}/queue_config")
        queue_config_dict = response.json()

    message_queue_config = _get_message_queue_config(queue_config_dict)
    message_queue_client = _get_message_queue_client(message_queue_config)

    service = WorkflowService(
        workflow=workflow,
        message_queue=message_queue_client,
        **workflow_config.model_dump(),
    )

    service_task = asyncio.create_task(service.launch_server())

    # let service spin up
    await asyncio.sleep(1)

    # register to message queue
    consumer_fn = await service.register_to_message_queue()

    # register to control plane
    control_plane_url = (
        f"http://{control_plane_config.host}:{control_plane_config.port}"
    )
    await service.register_to_control_plane(control_plane_url)

    # create consumer task
    consumer_task = asyncio.create_task(consumer_fn())

    # let things sync up
    await asyncio.sleep(1)

    all_tasks = [consumer_task, service_task]

    shutdown_handler = _get_shutdown_handler(all_tasks)
    loop = asyncio.get_event_loop()
    while loop.is_running():
        await asyncio.sleep(0.1)
        signal.signal(signal.SIGINT, shutdown_handler)

        for task in all_tasks:
            if task.done() and task.exception():  # type: ignore
                raise task.exception()  # type: ignore

In [27]:
from llama_agents import (
    AgentService,
    ToolService,
    MetaServiceTool,
    ControlPlaneServer,
    SimpleMessageQueue,
    AgentOrchestrator,
)

from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import ReActAgentWorker, ReActAgent



# create our multi-agent framework components
message_queue = SimpleMessageQueue()
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=llm),
)

# define Tool Service
tool_service = ToolService(
    message_queue=message_queue,
    tools=all_tools,
    running=True,
    step_interval=0.5,
)

# define meta-tools here
meta_tools = [
    await MetaServiceTool.from_tool_service(
        t.metadata.name,
        message_queue=message_queue,
        tool_service=tool_service,
    )
    for t in all_tools
]


# define Agent and agent service
# worker1 = FunctionCallingAgentWorker.from_tools(
worker1 = ReActAgentWorker.from_tools(
    meta_tools,
    llm=llm,
)
agent1 = worker1.as_agent()
agent_server_1 = AgentService(
    agent=agent1,
    message_queue=message_queue,
    description="Agent that answers questions based on the newly proposed SEC rule.",
    service_name="rule_proposal_agent",
)

In [ ]:
response = agent1.chat('When are written comments on this notice of joint proposed rulemaking need to be submitted?')
print(response.response)

In [ ]:
# questions
# Name of rule 
# Is it a proposed rule or a final rule
# Issue date and Federal Register date
# What agency(ies) is the Rule coming from
# If a proposed rule, when are public comments due by and where should they be sent (this info is in the Rule document under Dates and Addresses)

## Tree summarizer

In [ ]:
from llama_index.core.response_synthesizers import TreeSummarize
summarizer = TreeSummarize(llm=llm, verbose=True)
# prompt_summary = "You are a professional executive of AlphaTrAI. Your job is to summarize this text in great detail from a video transcription. The summary will be distributed to investors and stakeholders, so give a lot of details and examples from the transcription."
prompt_summary = f"""You are a professional executive at AlphaTrAI. Your job is to summarize the text from a video transcription. The summary will be a memo distributed to investors and stakeholders. Be sure it the memo has the following items:
1. Extract all the names of new hires and their position, and/or new advisors mentioned in the transcription.
2. Create a section to mention the personnel new to AlphaTrAI.
3. Include other highlights and progress made by AlphaTrAI.
4. Ensure the memo and ensure it is factual, optimistic, and any values mention come directly from the text. 

The transcription is as follows:\n{full_doc}"""

response = await summarizer.aget_response(prompt_summary, [doc.text for doc in documents])

In [ ]:
print(response)

## LLM direct summarization

In [ ]:
prompt_summary = f"""You are a professional executive at AlphaTrAI. Your job is to summarize the text from a video transcription. The summary will be a memo distributed to investors and stakeholders. Be sure it the memo has the following items:
1. Extract all the names of new hires and their position, and/or new advisors mentioned in the transcription.
2. Include other highlights and progress made by AlphaTrAI.
3. Ensure the memo is professional, fluid, factual, and optimistic. 

The transcription is as follows:\n{full_doc}"""

response = llm.complete(prompt_summary, max_tokens=5000)

In [ ]:
print(response.text)

## Agentic Summary

In [ ]:
!pip3 install llama-index-embeddings-huggingface llama-index-vector-stores-neo4jvector llama-index-graph-stores-neo4j
!apt install dialog apt-utils -y (done above)
!wget -O - https://debian.neo4j.com/neotechnology.gpg.key | gpg --dearmor -o /etc/apt/keyrings/neotechnology.gpg
!echo 'deb [signed-by=/etc/apt/keyrings/neotechnology.gpg] https://debian.neo4j.com stable latest' | tee -a /etc/apt/sources.list.d/neo4j.list
!apt list -a neo4j
!add-apt-repository universe -y
!apt install neo4j=1:5.22.0 -y
!echo "neo4j-enterprise neo4j/question select I ACCEPT" | debconf-set-selections
!echo "neo4j-enterprise neo4j/license note" | debconf-set-selections
!apt install openjdk-17-jre -y
!cd /var/lib/neo4j/plugins/ && wget https://github.com/neo4j/apoc/releases/download/5.22.0/apoc-5.22.0-core.jar

In [ ]:
set_neo4j_password('bewaretheneo')
add_lines_to_conf()


In [ ]:
from dotenv import load_dotenv
load_dotenv()

from llama_index.core.agent import ReActAgent
# from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from rag_utils import create_neo4j_graph_store, create_neo4j_graphrag, neo4j_query, set_neo4j_password, add_lines_to_conf
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import nest_asyncio
nest_asyncio.apply()



In [ ]:

llm.is_function_calling_model = True

embed_model_name = "Alibaba-NLP/gte-Qwen2-1.5B-instruct"
print("loading embed model...")
embed_model = HuggingFaceEmbedding(model_name=embed_model_name)

Settings.embed_model = embed_model
Settings.chunk_size = 300
Settings.chunk_overlap = 50

In [ ]:
from typing import Literal
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

entities = Literal["PEOPLE", 
                   "PLACE"
]

relations = Literal[
    "ROLE",
    "COMPANY"
]

validation_schema = {
    "People": ["ROLE"],
    "Place": ["COMPANY"],
}


In [ ]:
Settings.chunk_size = 300
Settings.chunk_overlap = 50

kg_extractor = SchemaLLMPathExtractor(
    llm=llm,
    possible_entities=entities,
    possible_relations=relations,
    kg_validation_schema=validation_schema,
    strict=True,  # if false, will allow triples outside of the schema
    num_workers=4,
    max_triplets_per_chunk=10,
)

graph_store = create_neo4j_graph_store(neo_url="bolt://localhost:7687", 
                                       password=os.getenv("NEO4J_PWD"), 
                                       config={"connection_timeout": 240, "connection_acquisition_timeout": 240, "max_connection_pool_size": 1000})
neo4j_query(graph_store, query="""MATCH (n) DETACH DELETE n""")


graph_index = create_neo4j_graphrag(documents, llm, embed_model, kg_extractor, graph_store)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = QueryEngineTool(
            query_engine=graph_index,
            metadata=ToolMetadata(
                name="graph_tool",
                description=(
                    "Useful for finding people names and roles, and the company they work for."
                ),
            ),
        ),


In [ ]:
!pip3 install llama-agents

In [ ]:
from llama_agents import (
    AgentService,
    ToolService,
    MetaServiceTool,
    ControlPlaneServer,
    SimpleMessageQueue,
    AgentOrchestrator,
)

from llama_index.core.agent import FunctionCallingAgentWorker, ReActAgentWorker, ReActAgent, LATSAgentWorker



# create our multi-agent framework components
message_queue = SimpleMessageQueue()
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=llm),
)

# define Tool Service
tool_service = ToolService(
    message_queue=message_queue,
    tools=[query_engine_tools],#, adding_tool],
    running=True,
    step_interval=0.5,
)

# define meta-tools here
meta_tools = [
    await MetaServiceTool.from_tool_service(
        t.metadata.name,
        message_queue=message_queue,
        tool_service=tool_service,
    )
    for t in [query_engine_tools]#, adding_tool]
]


# define Agent and agent service
# worker1 = FunctionCallingAgentWorker.from_tools(
worker1 = ReActAgentWorker.from_tools(
    meta_tools,
    llm=llm,
)

worker2 = LATSAgentWorker.from_tools(
    meta_tools,
    llm=llm,
    num_expansions=2,
    max_rollouts=3,
    verbose=True
)

agent1 = worker1.as_agent()
agent_server_1 = AgentService(
    agent=agent1,
    message_queue=message_queue,
    description="Summarize a transcription as a memo for investors and stakeholders.",
    service_name="summarize_transcription",
)